# Adds simple SQLAlchemy support to FastAPI
* pip install fastapi-sqlalchemy
* https://zzun.app/repo/mfreeborn-fastapi-sqlalchemy-python-fastapi-utilities

In [ ]:
from fastapi import FastAPI
from fastapi_sqlalchemy import DBSessionMiddleware  # middleware helper
# an object to provide global access to a database session
from fastapi_sqlalchemy import db

from app.models import User

app = FastAPI()

app.add_middleware(DBSessionMiddleware, db_url="sqlite://")

# once the middleware is applied, any route can then access the database session
# from the global ``db``


@app.get("/users")
def get_users():
    users = db.session.query(User).all()

    return users


## Usage outside of a route
* schedulers example

In [ ]:
import pytz
# other schedulers are available
from apscheduler.schedulers.asyncio import AsyncIOScheduler
from fastapi import FastAPI
from fastapi_sqlalchemy import db

from app.models import User, UserCount

app = FastAPI()

app.add_middleware(DBSessionMiddleware, db_url="sqlite://")


@app.on_event('startup')
async def startup_event():
    scheduler = AsyncIOScheduler(timezone=pytz.utc)
    scheduler.start()
    # runs every night at midnight
    scheduler.add_job(count_users_task, "cron", hour=0)


def count_users_task():
    """Count the number of users in the database and save it into the user_counts table."""

    # we are outside of a request context, therefore we cannot rely on ``DBSessionMiddleware``
    # to create a database session for us. Instead, we can use the same ``db`` object and
    # use it as a context manager, like so:

    with db():
        user_count = db.session.query(User).count()

        db.session.add(UserCount(user_count))
        db.session.commit()

    # no longer able to access a database session once the db() context manager has ended

    return users
